# Problems with list representation of polynomial
- negative exponents?
- fractional exponents?
- big exponents?
    - 1 + x^1000000000
- switch to a 'sparse' representation
- plug compatible with polylist
    - one of the advantages of using objects

In [2]:
import collections

class polydict:
    '''sparse poly representation using a dict
        sparse is {exponent:coefficient, ...}
        only non-zero terms appear in the dict
        
        {2:3, 1:2, 0:1} <=> 3*X**2 + 2*X + 1
    '''
    def __init__(self, d={}):

        # why the copy??
        self.sparse = collections.defaultdict(int)
        self.sparse.update(d)

    def printTerm(self, c ,e):
        ''' print a term'''
        cs = str(c)
        if c > 0:
            cs = '+ ' + cs
        if (e == 0):
            return(cs)
        if (e == 1):
            return('%s*X' % cs)    
        return('%s*X**%d' % (cs,e))   
        
    def __str__(self):
        if len(self.sparse) == 0:
            return('0')
        terms = [self.printTerm(self.sparse[e],e) 
                for e in sorted(self.sparse.keys(), 
                                reverse=True) 
                    if self.sparse[e] != 0]
        s = ' '.join(terms)
        if '+ ' == s[0:2]:
            s = s[2:]
        return (s)
    
    def __repr__(self):
        return(self.__str__())

    # don't allow a hash
    __hash__ = None  
    
    def __len__(self):
        return(len(self.sparse))

    # can explicity define bool
    def __bool__(self):
        return(False if len(self.sparse)==0 else True)
        
    def __iter__(self):
        # return a generator function that will
        # iterate thru (exp, coe) pairs
        return( (i for i in self.sparse.items() ))

    # should check types
    def __eq__(self, other):
        return(self.sparse == other.sparse)
        
    def __ne__(self, other):
        return(self.sparse != other.sparse)
        
    # define comparsion to be value of poly at 1
    def __lt__(self, other):
        return(self.evaluate(1) < other.evaluate(1))
        
    def __le__(self, other):
        return(self.evaluate(1) <= other.evaluate(1))
        
    # does poly 'contain' an exponent?
    def __contains__(self, e):
        return(e in self.sparse)

    def evaluate(self, n):
        '''eval poly at x=n'''
        sum = 0
        for e in self.sparse.keys():
            # self.sparse[e] is the coef
            sum += self.sparse[e]*n**e
        return(sum)
            
    def __add__(self, p2):
        '''add two polys'''
        n = self.sparse.copy()
        for k,v in p2.sparse.items():
            # defaultdict simplifies this
            n[k] += v
        return(polydict(n))
        
    def __getitem__(self, index):
        '''pull out terms of the poly
           p[2], p[2:5]
           '''
        keys = sorted(self.sparse.keys(), reverse=True)
        if isinstance(index, int):
            # if asked for a single term, p[n], index will
            # be an int
            inds = [index]
        if isinstance(index, slice):
            # if asked for a slice, p[n:m], index will be
            # a 'slice' object
            inds = range(*index.indices(len(keys)))
        d = {}
        for i in inds:
            e = keys[i]
            d[e] = self.sparse[e]
        return(polydict(d))
        
    def __rmul__(self, p2):
        ''' multiple by a scalar on the right
            5*p1
            '''
        if isinstance(p2, (int, float)):
            nd = {}
        for e,c in self.sparse.items():
            nd[e] = c * p2 
        return(polydict(nd))
        
    def differentiate(self):
        d = {}
        for e,c in self.sparse.items():
            if e != 0:
                d[e-1] = c * e
        return(polydict(d))
    
    def integrate(self):
        # doesn't handle log
        d = {}
        for e,c in self.sparse.items():
            d[e+1] = c / (e+1.)
        return(polydict(d))


In [3]:
    
p1 = polydict({2:3, 1:2, 0:1})
p2 = polydict({1:10, 2:5})

d = dict()

for n in range(1, 9):
        d[n] = 10 * n

p4 = polydict(d)

[p1, p2, p4]

[3*X**2 + 2*X + 1,
 5*X**2 + 10*X,
 80*X**8 + 70*X**7 + 60*X**6 + 50*X**5 + 40*X**4 + 30*X**3 + 20*X**2 + 10*X]

In [4]:
p1 + p2

8*X**2 + 12*X + 1

In [5]:
p4

80*X**8 + 70*X**7 + 60*X**6 + 50*X**5 + 40*X**4 + 30*X**3 + 20*X**2 + 10*X

In [6]:
# rmul

5 * p1

15*X**2 + 10*X + 5

In [7]:
p1.differentiate()

6*X + 2

In [8]:
p1.integrate()

1.0*X**3 + 1.0*X**2 + 1.0*X

In [1]:
# can test with 'sympy' symbolic package

import unittest
import sympy

X = sympy.symbols('X')
p1s = 3*X**2 + 2*X + 1
p2s = 5*X**2 + 10*X

def polys():
    print('p1: %s' % p1s)
    print('p2: %s' % p2s)
    print('p1+p2: %s' % (p1s + p2s))
    print('p1*p2: %s' % (p1s*p2s))
    print('p1*p2: %s' % (p1s*p2s).expand() )
    print('p1(3): %s' % p1s.subs(X, 3))
    print('d(p1)/dx %s' % p1s.diff(X))
    print('int(p1,x) %s' % p1s.integrate(X))


class SymPolyTest(unittest.TestCase):
    
    def testEval(self):
        self.assertEqual(sp2.subs(X,2), p2.evaluate(2))
        
# unittest.main()
        

In [2]:
polys()

p1: 3*X**2 + 2*X + 1
p2: 5*X**2 + 10*X
p1+p2: 8*X**2 + 12*X + 1
p1*p2: (5*X**2 + 10*X)*(3*X**2 + 2*X + 1)
p1*p2: 15*X**4 + 40*X**3 + 25*X**2 + 10*X
p1(3): 34
d(p1)/dx 6*X + 2
int(p1,x) X**3 + X**2 + X
